# Miniproyecto #3: Redes neuronales artificiales (ANN)


**Autores:**
*   Jorge Sebastián Arroyo Estrada CC. 1193482707
*   César Augusto Montoya Ocampo CC. 1036681523

**Tratamiento de Señales III**

**Facultad de Ingeniería**

**Universidad de Antioquia**

---

## Librerías

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

ModuleNotFoundError: No module named 'tensorflow.keras'

In [7]:
import tensorflow.keras as tf
print(tf.__version__)

ModuleNotFoundError: No module named 'tensorflow.keras'

## SVM
1. Cargar características (para ambos tamaños, con y sin PCA)
2. Realizar K-fold con K=5
3. Graficar métricas de clasificación
4. Graficar matriz de confusión

## Globales

In [1]:
# Tenemos varias opciones para la base de datos
DB_PATH64 = "../02_features/features_64.csv"
DB_PATH128 = "../02_features/features_128.csv"
DB_PCA_PATH64 = "../02_features/pca_64.csv"
DB_PCA_PATH128 = "../02_features/pca_128.csv"
LABELS_PATH = "../02_features/labels.csv"

DB_PATH = DB_PATH64

In [2]:
data = pd.read_csv(DB_PATH)
labels_df = pd.read_csv(LABELS_PATH)
X = data.iloc[:, :-1].values  # Assuming last column is the label
y = data.iloc[:, -1].values

# Map numeric classes to labels for better display in results
label_mapping = dict(zip(labels_df['Class'], labels_df['label']))
y = np.vectorize(label_mapping.get)(y)

# Define a function to create the model with variable parameters
def create_model(hidden_layers: tuple[int], activation: str, dropout_rate: float) -> tf.keras.models.Model:
    model = Sequential()
    # Input layer
    model.add(Dense(hidden_layers[0], activation=activation, input_shape=(X.shape[1],)))
    model.add(Dropout(dropout_rate))
    # Hidden layers
    for units in hidden_layers[1:]:
        model.add(Dense(units, activation=activation))
        model.add(Dropout(dropout_rate))
    # Output layer
    model.add(Dense(len(np.unique(y)), activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Wrapping the model for scikit-learn compatibility
model = KerasClassifier(
    model=create_model,
    hidden_layers=(128, 64),
    activation="relu",
    dropout_rate=0.3,
    epochs=20,
    batch_size=32,
    verbose=0
)

# Defining the parameter grid for GridSearch
param_grid = {
    "hidden_layers": [(256, 128, 64), (128, 64), (256, 128, 64, 32)],
    "activation": ["relu", "elu"],
    "dropout_rate": [0.3, 0.4]
}

# Set up StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(model, param_grid, cv=cv, scoring="accuracy", n_jobs=-1)
grid_search.fit(X, y)

# Print best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Evaluate on Cross-Validation with best model and display metrics
best_model = grid_search.best_estimator_
y_preds = cross_val_score(best_model, X, y, cv=cv, scoring='accuracy')
y_pred = best_model.predict(X)

print("Final Cross-Validation Accuracy:", y_preds.mean())
print("Classification Report:")
print(classification_report(y, y_pred))

# Confusion Matrix
cm = confusion_matrix(y, y_pred, labels=np.unique(y))
plt.figure(figsize=(10, 7))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(label_mapping))
plt.xticks(tick_marks, np.unique(y), rotation=45)
plt.yticks(tick_marks, np.unique(y))
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

# Displaying cross-validation metrics
print("Cross-Validation Scores:")
print(f"Accuracy: {accuracy_score(y, y_pred):.2f}")
print(f"Precision: {precision_score(y, y_pred, average='weighted'):.2f}")
print(f"F1-Score: {f1_score(y, y_pred, average='weighted'):.2f}")


NameError: name 'pd' is not defined

## Conclusiones
1. La característica de los histogramas de gradientes orientados (HOG) resultó útil para hallar las formas de los objetos en las imágenes, esto es especialmente significativo para la tarea de distinguir algunas frutas, por ejemplo de la forma elongada de una banana contra la forma circular de una naranja.
2. La característica de patrones binarios locales (LBP) sirvió para caracterizar las texturas presentes en las imágenes, siendo especialmente útil en las imágenes de "alta" resolución (128×128), ya que en las de baja resolución (64×64) se distorsiona demasiado.
3. Esta misma característica además permite visualizar las frutas, ya que algo tienen en común los fondos y es que son relativamente uniformes en textura, a diferencia de las frutas.
4. Mediante el análisis de histogramas de las componentes en el espacio Hue-Saturation-Value (HSV) se obtuvieron características adicionales que permiten aprovechar la diferencia en matiz, saturación y luminosidad de las imágenes, las cuales a simple vista otorgan información valiosa para la tarea de clasificación.